### **1. Librerias**

In [135]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access variables
email = os.getenv('email')
password = os.getenv('password')

print(f"Email: {email}")

Email: elianapinto20@outlook.com


In [136]:
# librerias 
import pandas as pd
import sys
import time


In [137]:
import warnings

# Ignorar warnings de pandas sobre cambios futuros
warnings.filterwarnings('ignore', category=FutureWarning)
import logging


In [138]:
# Librerias Selenium
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By  # buscar por etiquetas, clases, ids, paths, xpaths
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [139]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

### **2. Webscrapping data**

In [140]:
def pagina_principal(url):
    # agregar opciones al driver
    options = Options()

    # Ajustar el custom User-Agent, evitar bloqueos facilmente
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")

    # Inicializar el webdriver con las opciones personalizadas. 
    driver = webdriver.Chrome(options=options)

    # maximizar pantalla
    driver.maximize_window()

    # Entra a la pagina
    driver.get(url)
    return driver

In [141]:
def sign_in(driver):


    try:
        # Espera explícita para el enlace de inicio de sesión
        sign_in_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="signIn"]/div/div/a'))
        )
        sign_in_link.click()

        # Espera explícita para el botón de inicio de sesión
        sign_in_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.gr-button.gr-button--dark.gr-button--auth.authPortalConnectButton.authPortalSignInButton'))
        )
        sign_in_button.click()

        # Encuentra el campo de correo electrónico y envía el correo
        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_email"))
        )
        email_input.send_keys(f"{email}")

        # Encuentra el campo de contraseña y envía la contraseña
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_password"))
        )
        password_input.send_keys(f"{password}")

        # Encuentra y haz clic en el botón de envío
        sign_in_submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "signInSubmit"))
        )
        sign_in_submit_button.click()

    except Exception as e:
        print(f"Se produjo un error: {e}")


In [142]:
### NEW ! 
# def iniciar_y_logear(driver, url):
#     # Llamamos a la función para abrir la página
#     driver = iniciar_driver(url)
    
#     # Llamamos a la función para iniciar sesión después de abrir la página
#     sign_in(driver)

# # Llamamos a la función que incluye ambas acciones
# iniciar_y_logear(driver, 'http://example.com')


In [143]:
def go_to_all_genres(driver):
    
    # Espera hasta que el span con el texto "Browse ▾" sea clickeable
    browse_span = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Browse ▾']"))
    )

    # Haz clic en el span
    browse_span.click()

    # Espera hasta que el enlace "All Genres" sea clickeable
    all_genres_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[text()='All Genres']"))
    )

    # Haz clic en el enlace "All Genres"
    all_genres_link.click()

In [144]:
def create_dict():
    books_inf = {
    'books': [],
    'writer': [],
    'rating': [],
    'description': [],
    'statistics': [],
    'details': [],
    'urls': [],
    'gen': [],
    'reading': [],
    'will_read': [],
    'isbn': [],
    'ediciones': [],
    'published': [],
    'format': [],
    'asin': [],
    'language': []
    }
    return books_inf


In [145]:
def get_genres(driver):
    try:
        # Encuentra todos los nombres de géneros
        elements = driver.find_elements(By.CSS_SELECTOR, ".bigBoxContent a.gr-hyperlink")
        names = [element.text for element in elements]
    except TimeoutException:
        logging.error("Error general: No se pudo cargar la lista de géneros (Timeout).")
    except NoSuchElementException:
        logging.error("Error: No se encontraron los géneros en la página.")
    except WebDriverException as e:
        logging.error(f"Error del navegador")
    except Exception as e:
        logging.error(f"Error inesperado: {e}")

    return names[5:6]

In [146]:
# //[@class="actionLink" and contains(text(), "More {gen} books...")]
        
        ## AQUI VA UNA FUNCION QUE BUSCA LOS LIBROS Y LUEGO LOS REVISA Y SACA LA INFO 
        ## DEBO VOLVER ATRAS PORQUE TIENE VARIOS LINKS LA PAGINA O? 


        # # Regresar nuevamente a la página anterior
        # driver.back()
        # time.sleep(2)  # Pausa para permitir la carga de la página

In [147]:
def select_a_genre(driver, genres):

    for gen in genres:
        # Este rango parece estar limitado a 1 género, puedes ajustar el rango según sea necesario
        logging.info(f"Procesando género: {gen}")
        
        try:
            # Esperar que el enlace sea clickeable y hacer clic
            art_link = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, f"//a[contains(text(), '{gen}')]"))
            )
            art_link.click()
            time.sleep(1)  # Pausa para permitir la carga de la página
            
            logging.info(f"Navegando a: {driver.current_url}")
            return gen
        

        except TimeoutException:
            logging.error(f"Error: No se pudo hacer clic en el enlace para '{gen}' (Timeout).")
        except NoSuchElementException:
            logging.error(f"Error: No se encontró el enlace para '{gen}'.")
        except WebDriverException as e:
            logging.error(f"Error del navegador: {e}")


In [148]:
# def get_links(driver): 
#     try: 
#         logging.info("Proceeding with scraping book URLs...")
#         # Usamos un conjunto para evitar duplicados
#         URLs = set()
        
#         # Encontramos los elementos <a> que empiezan con '/book/show'
#         anchor_tags = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/book/show')]")
        
#         # Iteramos sobre los elementos encontrados
#         for tag in anchor_tags:
#             url = tag.get_attribute('href')
#             if url in URLs:
#                 logging.info(f"Duplicate URL skipped: {url}")
#             else:
#                 URLs.add(url)
#                 logging.info(f"Found unique book URL: {url}")
        
#         # Convertimos el conjunto en una lista para devolverlo
#         return list(URLs)
    
#     except Exception as e:
#         logging.error("Error occurred while trying to find book URLs", exc_info=True)
#         return []


In [149]:
# def scroll_until_found(driver, xpath):
#     for _ in range(10):  # Intenta 10 desplazamientos como máximo
#         try:
#             button = driver.find_element(By.XPATH, xpath)
#             driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
#             return button
#         except NoSuchElementException:
#             driver.execute_script("window.scrollBy(0, 500);")
#             time.sleep(1)  # Espera para permitir que se cargue más contenido
#     raise Exception(f"No se encontró el elemento con XPath: {xpath}")

# def explore_books(driver, gen): 
#     try:
#         gen_formatted = gen.lower()  # Convierte "art" a "Art"
#         xpath = f'//div[@class="moreLink"]/a[contains(text(), "More {gen_formatted} books...")]'
#         button = scroll_until_found(driver, xpath)
#         button.click()
#         url_books = get_links(driver)
#         print(f"Botón 'More {gen} books...' clickeado exitosamente.")
#     except Exception as e:
#         print(f"No se pudo hacer clic en el botón: {e}")
    
#     return url_books


In [150]:
# logging.info(f"Navegando a: {driver.current_url}")

# # Ahora buscar los enlaces en esta página
# urls = get_links()
# for url in urls:
#     get_information(url, gen)

In [151]:
url = "https://www.goodreads.com" 

In [152]:
driver = pagina_principal(url)

In [153]:
sign_in(driver)

In [154]:
# go_to_all_genres(driver)

In [155]:
dict_dataframe = create_dict()

In [156]:
dict_dataframe

{'books': [],
 'writer': [],
 'rating': [],
 'description': [],
 'statistics': [],
 'details': [],
 'urls': [],
 'gen': [],
 'reading': [],
 'will_read': [],
 'isbn': [],
 'ediciones': [],
 'published': [],
 'format': [],
 'asin': [],
 'language': []}

In [157]:
# gen = select_a_genre(driver, genres)

In [158]:
# gen

In [159]:
# explore_books(driver, gen)

In [160]:
books_links = pd.read_csv('../reviews_goodreads_tarea.csv')['book_id']
urls = list(set(list(books_links)))
# urls

In [161]:
def create_link_books(urls):
    urls_books_complete = []
    url_base = 'https://www.goodreads.com/book/show/'
    for i in urls:
        url_complete = f"{url_base}{i}"
        urls_books_complete.append(url_complete)
    return urls_books_complete


urls_to_scrap = create_link_books(urls)

In [162]:
# urls_to_scrap

In [163]:
def get_information(driver, url):
    dict_dataframe['urls'].append(url)

    try:
        logging.info(f"Processing URL: {url}")  # Log the URL being processed

        if url:
            # Navigate to the book's page
            driver.get(url)
            
            # Extract book name
            try:
                names_book = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.Text.Text__title1')))
                name = names_book.text.strip()
                dict_dataframe['books'].append(name if name else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting book name from {url}: {e}")

            # Extract authors
            try:
                autores = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.ContributorLink__name')))
                name_writer = [autor.text.strip() for autor in autores if autor.text.strip()]
                dict_dataframe['writer'].append(name_writer if name_writer else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting authors from {url}: {e}")

            # Extract rating
            try:
                ratings_num = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.RatingStatistics__rating')))
                ratings_num_text = ratings_num.text.strip()
                dict_dataframe['rating'].append(ratings_num_text if ratings_num_text else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting rating from {url}: {e}")

            # Extract description
            try:
                desc = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.Formatted')))
                description = desc.text.strip()
                dict_dataframe['description'].append(description if description else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting description from {url}: {e}")

            # Extract statistics
            try:
                statics = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'RatingStatistics__meta')))
                ratings_book_views = statics[0].get_attribute('aria-label').strip() if statics else None
                dict_dataframe['statistics'].append(ratings_book_views if ratings_book_views else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting statistics from {url}: {e}")

            # Extract additional details
            try:
                detail = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.FeaturedDetails p')))
                dets = [det.text.strip() for det in detail if det.text.strip()]
                dict_dataframe['details'].append(dets if dets else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting details from {url}: {e}")

            # Wait for the "Book details and editions" button and click it
            try:
                button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button--inline') and contains(@aria-label, 'Book details and editions')]"))
                )
                button.click()
                logging.info("Botón de detalles encontrado y clickeado.")
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error finding or clicking the button: {e}")

            # Wait for the elements that contain 'Format', 'Published', 'ASIN', and 'Language'
            try:
                # Wait for the "Format" element to be visible
                format_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='DescListItem'][contains(., 'Format')]/dd//div[@class='TruncatedContent__text']"))
                )
                dict_dataframe['format'].append(format_element.text.strip() if format_element else None)

                # Wait for the "Published" element to be visible
                published_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='DescListItem'][contains(., 'Published')]/dd//div[@class='TruncatedContent__text']"))
                )
                dict_dataframe['published'].append(published_element.text.strip() if published_element else None)

                # Wait for the "ASIN" element to be visible
                asin_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='DescListItem'][contains(., 'ASIN')]/dd//span[@data-testid='asin']"))
                )
                dict_dataframe['asin'].append(asin_element.text.strip() if asin_element else None)

                # Wait for the "Language" element to be visible
                language_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='DescListItem'][contains(., 'Language')]/dd//div[@class='TruncatedContent__text']"))
                )
                dict_dataframe['language'].append(language_element.text.strip() if language_element else None)

            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting data: {e}")


                        # Extract additional details after clicking the button
            try:
                # Waiting for the element containing "Show all" to be visible
                element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Show all')]"))
                )
                texto = element.text
                if texto:
                    print(f"Texto encontrado: {texto}")  # Mostrar el texto en la consola
                    dict_dataframe['ediciones'].append(texto)  # Añadir el texto a la lista
                else:
                    logging.info("El elemento está presente pero no contiene texto.")
                    dict_dataframe['ediciones'].append(None)

            except TimeoutException:
                logging.warning("El elemento 'Show all' no está presente en esta URL.")
                dict_dataframe['ediciones'].append(None)  # Registrar que no había texto

            except NoSuchElementException as e:
                logging.error(f"Error al buscar el texto: {e}")
                dict_dataframe['ediciones'].append(None)  # Registrar que no había texto
    except Exception as e:
        logging.error(f"An unexpected error occurred for URL {url}: {e}")



In [164]:
for i in urls_to_scrap[0:4]:
    get_information(driver, i)

ERROR:root:Error extracting data: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6560F80D5+2992373]
	(No symbol) [0x00007FF655D8BFD0]
	(No symbol) [0x00007FF655C2590A]
	(No symbol) [0x00007FF655C7926E]
	(No symbol) [0x00007FF655C7955C]
	(No symbol) [0x00007FF655CC27D7]
	(No symbol) [0x00007FF655C9F3AF]
	(No symbol) [0x00007FF655CBF584]
	(No symbol) [0x00007FF655C9F113]
	(No symbol) [0x00007FF655C6A918]
	(No symbol) [0x00007FF655C6BA81]
	GetHandleVerifier [0x00007FF656156A2D+3379789]
	GetHandleVerifier [0x00007FF65616C32D+3468109]
	GetHandleVerifier [0x00007FF656160043+3418211]
	GetHandleVerifier [0x00007FF655EEC78B+847787]
	(No symbol) [0x00007FF655D9757F]
	(No symbol) [0x00007FF655D92FC4]
	(No symbol) [0x00007FF655D9315D]
	(No symbol) [0x00007FF655D82979]
	BaseThreadInitThunk [0x00007FF939487374+20]
	RtlUserThreadStart [0x00007FF93A35CC91+33]

ERROR:root:Error extracting data: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6560F80D5+2992373]
	(No symbol) [0x00007FF655D8BFD0]
	

Texto encontrado: Show all 283 editions


ERROR:root:Error finding or clicking the button: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6560F80D5+2992373]
	(No symbol) [0x00007FF655D8BFD0]
	(No symbol) [0x00007FF655C2590A]
	(No symbol) [0x00007FF655C7926E]
	(No symbol) [0x00007FF655C7955C]
	(No symbol) [0x00007FF655CC27D7]
	(No symbol) [0x00007FF655C9F3AF]
	(No symbol) [0x00007FF655CBF584]
	(No symbol) [0x00007FF655C9F113]
	(No symbol) [0x00007FF655C6A918]
	(No symbol) [0x00007FF655C6BA81]
	GetHandleVerifier [0x00007FF656156A2D+3379789]
	GetHandleVerifier [0x00007FF65616C32D+3468109]
	GetHandleVerifier [0x00007FF656160043+3418211]
	GetHandleVerifier [0x00007FF655EEC78B+847787]
	(No symbol) [0x00007FF655D9757F]
	(No symbol) [0x00007FF655D92FC4]
	(No symbol) [0x00007FF655D9315D]
	(No symbol) [0x00007FF655D82979]
	BaseThreadInitThunk [0x00007FF939487374+20]
	RtlUserThreadStart [0x00007FF93A35CC91+33]

ERROR:root:Error extracting data: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6560F80D5+2992373]
	(No symbol) [0x0000

Texto encontrado: Show all 1355 editions


ERROR:root:Error finding or clicking the button: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6560F80D5+2992373]
	(No symbol) [0x00007FF655D8BFD0]
	(No symbol) [0x00007FF655C2590A]
	(No symbol) [0x00007FF655C7926E]
	(No symbol) [0x00007FF655C7955C]
	(No symbol) [0x00007FF655CC27D7]
	(No symbol) [0x00007FF655C9F3AF]
	(No symbol) [0x00007FF655CBF584]
	(No symbol) [0x00007FF655C9F113]
	(No symbol) [0x00007FF655C6A918]
	(No symbol) [0x00007FF655C6BA81]
	GetHandleVerifier [0x00007FF656156A2D+3379789]
	GetHandleVerifier [0x00007FF65616C32D+3468109]
	GetHandleVerifier [0x00007FF656160043+3418211]
	GetHandleVerifier [0x00007FF655EEC78B+847787]
	(No symbol) [0x00007FF655D9757F]
	(No symbol) [0x00007FF655D92FC4]
	(No symbol) [0x00007FF655D9315D]
	(No symbol) [0x00007FF655D82979]
	BaseThreadInitThunk [0x00007FF939487374+20]
	RtlUserThreadStart [0x00007FF93A35CC91+33]

ERROR:root:Error extracting data: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6560F80D5+2992373]
	(No symbol) [0x0000

Texto encontrado: Show all 707 editions


In [165]:
dict_dataframe



{'books': ['Harry Potter and the Half-Blood Prince',
  'Good Omens: The Nice and Accurate Prophecies of Agnes Nutter, Witch',
  "Harry Potter and the Sorcerer's Stone",
  'Harry Potter and the Order of the Phoenix'],
 'writer': [['J.K. Rowling', 'J.K. Rowling'],
  ['Terry Pratchett', 'Neil Gaiman', 'Terry Pratchett'],
  ['J.K. Rowling', 'J.K. Rowling'],
  ['J.K. Rowling', 'Mary GrandPré', 'J.K. Rowling']],
 'rating': ['4.58', '4.26', '4.47', '4.50'],
 'description': ["It is the middle of the summer, but there is an unseasonal mist pressing against the windowpanes. Harry Potter is waiting nervously in his bedroom at the Dursleys' house in Privet Drive for a visit from Professor Dumbledore himself. One of the last times he saw the Headmaster, he was in a fierce one-to-one duel with Lord Voldemort, and Harry can't quite believe that Professor Dumbledore will actually appear at the Dursleys' of all places. Why is the Professor coming to visit him now? What is it that cannot wait until Harr